In [ ]:
# Copyright 2021 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Vertex SDK：为批量预测训练图像对象检测模型

<table align="left">
  <td>
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/vertex-ai-samples/tree/master/notebooks/official/automl/sdk_automl_image_object_detection_batch.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo"> 在 Colab 中运行
    </a>
  </td>
  <td>
    <a href="https://github.com/GoogleCloudPlatform/vertex-ai-samples/tree/master/notebooks/official/automl/sdk_automl_image_object_detection_batch.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      在 GitHub 上查看
    </a>
  </td>
  <td>
    <a href="https://console.cloud.google.com/ai/platform/notebooks/deploy-notebook?download_url=https://github.com/GoogleCloudPlatform/vertex-ai-samples/tree/master/notebooks/official/automl/sdk_automl_image_object_detection_batch.ipynb">
      在 Google Cloud 笔记本中打开
    </a>
  </td>
</table>
<br/><br/><br/>

## 概述

本教程演示了如何使用Vertex SDK来创建图像物体检测模型，并使用Google Cloud [AutoML](https://cloud.google.com/vertex-ai/docs/start/automl-users)模型进行批量预测。

### 数据集

本教程使用的数据集是来自[TensorFlow数据集](https://www.tensorflow.org/datasets/catalog/open_images_v4)中的沙拉类别。这个数据集不需要任何特征工程。本教程中将使用的数据集版本存储在公共云存储桶中。训练模型会预测图像中五种项目类别中的沙拉、海鲜、番茄、烘焙食品或奶酪的边界框位置和对应类型。

### 目标

在本教程中，您将从 Python 脚本中创建一个 AutoML 图像对象检测模型，然后使用 Vertex SDK 进行批量预测。您也可以选择使用 `gcloud` 命令行工具或在 Cloud Console 上在线创建和部署模型。

执行的步骤包括：

- 创建一个 Vertex `Dataset` 资源。
- 训练模型。
- 查看模型评估。
- 进行批量预测。

使用批量预测和在线预测之间有一个关键区别：

* 预测服务：对整个数据集（即一个或多个数据项）进行按需预测，并实时返回结果。

* 批量预测服务：对整个数据集进行排队（批量）预测，并在准备好时将结果存储在 Cloud Storage 存储桶中。

### 成本

本教程使用 Google Cloud 中可计费的组件：

* Vertex AI
* 云存储

了解 [Vertex AI 价格](https://cloud.google.com/vertex-ai/pricing) 和 [云存储价格](https://cloud.google.com/storage/pricing)，并使用 [价格计算器](https://cloud.google.com/products/calculator/) 根据您预期的使用量生成成本估算。

### 设置本地开发环境

如果您正在使用Colab或Google Cloud笔记本，则您的环境已经符合运行此笔记本所需的所有要求。您可以跳过此步骤。

否则，请确保您的环境符合此笔记本的要求。您需要以下内容：

- 云存储SDK
- Git
- Python 3
- virtualenv
- 在使用Python 3的虚拟环境中运行的Jupyter笔记本

[设置Python开发环境](https://cloud.google.com/python/setup)和[Jupyter安装指南](https://jupyter.org/install)提供了满足这些要求的详细说明。以下步骤提供了一套简化的指令：

1. [安装并初始化SDK](https://cloud.google.com/sdk/docs/)。

2. [安装Python 3](https://cloud.google.com/python/setup#installing_python)。

3. [安装virtualenv](https://cloud.google.com/python/setup#installing_and_using_virtualenv)并创建一个使用Python 3的虚拟环境。激活虚拟环境。

4. 要安装Jupyter，在终端shell中运行 `pip3 install jupyter`。

5. 要启动Jupyter，在终端shell中运行 `jupyter notebook`。

6. 在Jupyter Notebook仪表板中打开此笔记本。

安装

安装最新版本的Python的Vertex SDK。

In [ ]:
import os

# Google Cloud Notebook
if os.path.exists("/opt/deeplearning/metadata/env_version"):
    USER_FLAG = "--user"
else:
    USER_FLAG = ""

! pip3 install --upgrade google-cloud-aiplatform $USER_FLAG

安装最新的GA版本*google-cloud-storage*库。

In [ ]:
! pip3 install -U google-cloud-storage $USER_FLAG

In [ ]:
if os.environ["IS_TESTING"]:
    ! pip3 install --upgrade tensorflow $USER_FLAG

重新启动内核

安装了额外的软件包后，您需要重新启动笔记本内核，以便它可以找到这些软件包。

In [ ]:
import os

if not os.getenv("IS_TESTING"):
    # Automatically restart kernel after installs
    import IPython

    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)

## 在开始之前

### GPU运行时

本教程不需要 GPU 运行时。

### 设置 Google Cloud 项目

**以下步骤是必需的，无论您使用什么样的笔记本环境。**

1. [选择或创建一个 Google Cloud 项目](https://console.cloud.google.com/cloud-resource-manager)。当您首次创建帐户时，您将获得 $300 的免费信用额以用于计算/存储成本。

2. [确保您的项目已启用计费。](https://cloud.google.com/billing/docs/how-to/modify-project)

3. [启用以下 API：Vertex AI API、Compute Engine API 和 Cloud Storage。](https://console.cloud.google.com/flows/enableapi?apiid=ml.googleapis.com,compute_component,storage-component.googleapis.com)

4. 如果您在本地运行此笔记本，您需要安装 [Cloud SDK](https://cloud.google.com/sdk)。

5. 在下面的单元格中输入您的项目 ID。然后运行该单元格以确保 Cloud SDK 为本笔记本中的所有命令使用正确的项目。

**注意**：Jupyter 运行以 `!` 为前缀的行作为 shell 命令，并且它会对以 `$` 为前缀的 Python 变量进行插值。

In [ ]:
PROJECT_ID = "[your-project-id]"  # @param {type:"string"}

In [ ]:
if PROJECT_ID == "" or PROJECT_ID is None or PROJECT_ID == "[your-project-id]":
    # Get your GCP project id from gcloud
    shell_output = ! gcloud config list --format 'value(core.project)' 2>/dev/null
    PROJECT_ID = shell_output[0]
    print("Project ID:", PROJECT_ID)

In [ ]:
! gcloud config set project $PROJECT_ID

区域

您还可以更改“REGION”变量，在整个笔记本的其余部分中使用。以下是Vertex AI支持的区域。我们建议您选择最接近您的区域。

- 美洲：`us-central1`
- 欧洲：`europe-west4`
- 亚太地区：`asia-east1`

您无法使用多区域存储桶进行Vertex AI的训练。并非所有区域都支持所有Vertex AI服务。

了解更多关于[Vertex AI区域](https://cloud.google.com/vertex-ai/docs/general/locations)

In [ ]:
REGION = "us-central1"  # @param {type: "string"}

时间戳

如果您在进行实时教程会话，则可能正在使用共享的测试帐户或项目。为了避免在创建的资源上发生名称冲突，您需要为每个实例会话创建一个时间戳，并将该时间戳附加到您在本教程中创建的资源名称上。

In [ ]:
from datetime import datetime

TIMESTAMP = datetime.now().strftime("%Y%m%d%H%M%S")

### 验证您的Google Cloud帐户

**如果您正在使用Google Cloud笔记本**，您的环境已经经过验证。跳过这一步。

**如果您正在使用Colab**，运行下面的单元格，并按提示进行身份验证，通过oAuth验证您的帐户。

**否则**，请按照以下步骤操作：

在Cloud控制台中，转到[创建服务帐户密钥](https://console.cloud.google.com/apis/credentials/serviceaccountkey)页面。

**点击创建服务帐户**。

在**服务帐户名称**字段中，输入名称，然后点击**创建**。

在**授予此服务帐户对项目的访问权限**部分，点击角色下拉列表。在过滤框中输入“Vertex”，并选择**Vertex管理员**。在过滤框中输入“Storage Object Admin”，并选择**存储对象管理员**。

点击创建。包含您的密钥的JSON文件将下载到您的本地环境。

在下面的单元格中，输入您的服务帐户密钥的路径作为GOOGLE_APPLICATION_CREDENTIALS变量，然后运行该单元格。

In [ ]:
# If you are running this notebook in Colab, run this cell and follow the
# instructions to authenticate your GCP account. This provides access to your
# Cloud Storage bucket and lets you submit training jobs and prediction
# requests.

import os
import sys

# If on Google Cloud Notebook, then don't execute this code
if not os.path.exists("/opt/deeplearning/metadata/env_version"):
    if "google.colab" in sys.modules:
        from google.colab import auth as google_auth

        google_auth.authenticate_user()

    # If you are running this notebook locally, replace the string below with the
    # path to your service account key and run this cell to authenticate your GCP
    # account.
    elif not os.getenv("IS_TESTING"):
        %env GOOGLE_APPLICATION_CREDENTIALS ''

创建一个云存储桶

**无论您是在哪个笔记本环境中，都需要执行以下步骤。**

当您初始化用于 Python 的 Vertex SDK 时，您需要指定一个云存储暂存桶。暂存桶是您的数据集和模型资源的所有关联数据在会话之间保留的地方。

在下方设置您的云存储桶的名称。存储桶的名称必须在所有谷歌云项目中全局唯一，包括您的组织之外的项目。

In [ ]:
BUCKET_NAME = "gs://[your-bucket-name]"  # @param {type:"string"}

In [ ]:
if BUCKET_NAME == "" or BUCKET_NAME is None or BUCKET_NAME == "gs://[your-bucket-name]":
    BUCKET_NAME = "gs://" + PROJECT_ID + "aip-" + TIMESTAMP

只有在您的存储桶尚不存在时才运行以下单元格以创建您的云存储存储桶。

In [ ]:
! gsutil mb -l $REGION $BUCKET_NAME

最后，通过检查Cloud Storage存储桶的内容来验证访问权限。

In [ ]:
! gsutil ls -al $BUCKET_NAME

### 设置变量

接下来，设置一些在教程中使用的变量。
### 导入库并定义常量

In [ ]:
import google.cloud.aiplatform as aip

初始化用于Python的Vertex SDK

初始化用于您的项目和相应存储桶的Python Vertex SDK。

In [ ]:
aip.init(project=PROJECT_ID, staging_bucket=BUCKET_NAME)

# 教程

现在您已经准备好开始创建自己的 AutoML 图像物体检测模型。

云存储训练数据的位置。

现在将变量`IMPORT_FILE`设置为存储在云存储中的CSV索引文件的位置。

In [ ]:
IMPORT_FILE = "gs://cloud-samples-data/vision/salads.csv"

快速查看您的数据

本教程使用存储在公共云存储桶中的一版本沙拉数据集，使用CSV索引文件。

首先，快速查看数据。通过计算CSV索引文件中的行数（`wc -l`）来计算示例的数量，然后查看前几行。

In [ ]:
if "IMPORT_FILES" in globals():
    FILE = IMPORT_FILES[0]
else:
    FILE = IMPORT_FILE

count = ! gsutil cat $FILE | wc -l
print("Number of Examples", int(count[0]))

print("First 10 rows")
! gsutil cat $FILE | head

### 创建数据集

接下来，使用`ImageDataset`类的`create`方法创建`Dataset`资源，需要提供以下参数：

- `display_name`：`Dataset`资源的人类可读名称。
- `gcs_source`：一个或多个数据集索引文件的列表，用于将数据项导入`Dataset`资源中。
- `import_schema_uri`：数据项的数据标签模式。

这个操作可能需要几分钟时间。

In [ ]:
dataset = aip.ImageDataset.create(
    display_name="Salads" + "_" + TIMESTAMP,
    gcs_source=[IMPORT_FILE],
    import_schema_uri=aip.schema.dataset.ioformat.image.bounding_box,
)

print(dataset.resource_name)

### 创建和运行训练管道

要训练一个AutoML模型，您需要执行两个步骤：1）创建训练管道，2）运行管道。

#### 创建训练管道

使用`AutoMLImageTrainingJob`类创建一个AutoML训练管道，具有以下参数：

- `display_name`：`TrainingJob`资源的人类可读名称。
- `prediction_type`：训练模型的任务类型。
  - `classification`：图像分类模型。
  - `object_detection`：图像目标检测模型。
- `multi_label`：如果是分类任务，则是单标签 (`False`) 还是多标签 (`True`)。
- `model_type`：部署的模型类型。
  - `CLOUD`：部署在谷歌云上。
  - `CLOUD_HIGH_ACCURACY_1`：优化部署在谷歌云上的准确性。
  - `CLOUD_LOW_LATENCY_`：优化部署在谷歌云上的延迟。
  - `MOBILE_TF_VERSATILE_1`：部署在边缘设备上。
  - `MOBILE_TF_HIGH_ACCURACY_1`：优化在边缘设备上的准确性。
  - `MOBILE_TF_LOW_LATENCY_1`：优化在边缘设备上的延迟。
- `base_model`：（可选）从现有的`Model`资源进行迁移学习 -- 仅支持图像分类。

实例化的对象是训练作业的DAG（有向无环图）。

In [ ]:
dag = aip.AutoMLImageTrainingJob(
    display_name="salads_" + TIMESTAMP,
    prediction_type="object_detection",
    multi_label=False,
    model_type="CLOUD",
    base_model=None,
)

print(dag)

#### 运行训练流水线

接下来，您可以通过调用方法 `run` 运行DAG来启动训练作业，参数如下：

- `dataset`：用于训练模型的 `Dataset` 资源。
- `model_display_name`：训练模型的可读名称。
- `training_fraction_split`：用于训练的数据集百分比。
- `test_fraction_split`：用于测试（留出数据）的数据集百分比。
- `validation_fraction_split`：用于验证的数据集百分比。
- `budget_milli_node_hours`：（可选）指定的最大训练时间，以毫小时为单位（1000 = 小时）。
- `disable_early_stopping`：如果为 `True`，如果服务认为无法进一步改善模型的目标测量，则训练可能在使用整个预算之前完成。

完成 `run` 方法后，将返回 `Model` 资源。

训练流水线的执行将需要最多60分钟。

In [ ]:
model = dag.run(
    dataset=dataset,
    model_display_name="salads_" + TIMESTAMP,
    training_fraction_split=0.8,
    validation_fraction_split=0.1,
    test_fraction_split=0.1,
    budget_milli_node_hours=20000,
    disable_early_stopping=False,
)

## 查看模型评估分数
在模型训练完成后，您可以查看其评估分数。

首先，您需要获取对新模型的引用。与数据集一样，您可以使用部署模型时创建的模型变量的引用，也可以列出项目中的所有模型。

In [ ]:
# Get model resource ID
models = aip.Model.list(filter="display_name=salads_" + TIMESTAMP)

# Get a reference to the Model Service client
client_options = {"api_endpoint": f"{REGION}-aiplatform.googleapis.com"}
model_service_client = aip.gapic.ModelServiceClient(client_options=client_options)

model_evaluations = model_service_client.list_model_evaluations(
    parent=models[0].resource_name
)
model_evaluation = list(model_evaluations)[0]
print(model_evaluation)

发送一个批量预测请求

向部署的模型发送批量预测。

### 获取测试项目

现在对您的 Vertex 模型进行批量预测。您将使用数据集中的任意示例作为测试项目。不必担心这些示例可能已在训练模型中使用过，我们只是想演示如何进行预测。

In [ ]:
test_items = !gsutil cat $IMPORT_FILE | head -n2
cols_1 = str(test_items[0]).split(",")
cols_2 = str(test_items[1]).split(",")
if len(cols_1) == 11:
    test_item_1 = str(cols_1[1])
    test_label_1 = str(cols_1[2])
    test_item_2 = str(cols_2[1])
    test_label_2 = str(cols_2[2])
else:
    test_item_1 = str(cols_1[0])
    test_label_1 = str(cols_1[1])
    test_item_2 = str(cols_2[0])
    test_label_2 = str(cols_2[1])

print(test_item_1, test_label_1)
print(test_item_2, test_label_2)

### 复制测试项

对于批量预测，请将测试项复制到您的云存储桶中。

In [ ]:
file_1 = test_item_1.split("/")[-1]
file_2 = test_item_2.split("/")[-1]

! gsutil cp $test_item_1 $BUCKET_NAME/$file_1
! gsutil cp $test_item_2 $BUCKET_NAME/$file_2

test_item_1 = BUCKET_NAME + "/" + file_1
test_item_2 = BUCKET_NAME + "/" + file_2

### 制作批量输入文件

现在制作一个批量输入文件，您将把它存储在您的本地云存储桶中。批量输入文件可以是CSV或JSONL格式。在本教程中，您将使用JSONL格式。对于JSONL文件，您每行制作一个字典条目，用于每个数据项（实例）。字典包含以下键/值对：

- `content`：图像的云存储路径。
- `mime_type`：内容类型。在我们的示例中，它是一个`jpeg`文件。

例如：

                        {'content': '[your-bucket]/file1.jpg', 'mime_type': 'jpeg'}

In [ ]:
import json

import tensorflow as tf

gcs_input_uri = BUCKET_NAME + "/test.jsonl"
with tf.io.gfile.GFile(gcs_input_uri, "w") as f:
    data = {"content": test_item_1, "mime_type": "image/jpeg"}
    f.write(json.dumps(data) + "\n")
    data = {"content": test_item_2, "mime_type": "image/jpeg"}
    f.write(json.dumps(data) + "\n")

print(gcs_input_uri)
! gsutil cat $gcs_input_uri

### 进行批量预测请求

现在您的模型资源已经训练好了，您可以通过调用`batch_predict()`方法进行批量预测，传入以下参数：

- `job_display_name`：批量预测作业的人类可读名称。
- `gcs_source`：一个或多个批量请求输入文件的列表。
- `gcs_destination_prefix`：用于存储批量预测结果的Cloud Storage位置。
- `sync`：如果设置为True，调用将在等待异步批处理作业完成时阻塞。

In [ ]:
batch_predict_job = model.batch_predict(
    job_display_name="salads_" + TIMESTAMP,
    gcs_source=gcs_input_uri,
    gcs_destination_prefix=BUCKET_NAME,
    sync=False,
)

print(batch_predict_job)

### 等待批量预测作业完成

接下来，等待批量作业完成。或者，可以在`batch_predict()`方法中将参数`sync`设置为`True`，以阻塞直到批量预测作业完成。

In [ ]:
batch_predict_job.wait()

### 获取预测结果

接下来，从已完成的批量预测作业中获取结果。

结果将写入您在批量预测请求中指定的云存储输出桶中。您可以调用方法iter_outputs()以获取生成结果时的每个云存储文件的列表。每个文件以 JSON 格式包含一个或多个预测请求：

- `content`：预测请求。
- `prediction`：预测响应。
 - `ids`：每个预测请求的内部分配的唯一标识符。
 - `displayNames`：每个类别标签的类名称。
 - `bboxes`：每个检测到的对象的边界框。

In [ ]:
import json

import tensorflow as tf

bp_iter_outputs = batch_predict_job.iter_outputs()

prediction_results = list()
for blob in bp_iter_outputs:
    if blob.name.split("/")[-1].startswith("prediction"):
        prediction_results.append(blob.name)

tags = list()
for prediction_result in prediction_results:
    gfile_name = f"gs://{bp_iter_outputs.bucket.name}/{prediction_result}"
    with tf.io.gfile.GFile(name=gfile_name, mode="r") as gfile:
        for line in gfile.readlines():
            line = json.loads(line)
            print(line)
            break

清理

要清理此项目中使用的所有Google Cloud资源，您可以删除用于本教程的[Google Cloud项目](https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects)。

否则，您可以删除本教程中创建的各个资源：

- 数据集
- 流水线
- 模型
- 终端
- AutoML训练作业
- 批处理作业
- 自定义作业
- 超参数调整作业
- Cloud Storage存储桶

In [ ]:
delete_all = True

if delete_all:
    # Delete the dataset using the Vertex dataset object
    try:
        if "dataset" in globals():
            dataset.delete()
    except Exception as e:
        print(e)

    # Delete the model using the Vertex model object
    try:
        if "model" in globals():
            model.delete()
    except Exception as e:
        print(e)

    # Delete the endpoint using the Vertex endpoint object
    try:
        if "endpoint" in globals():
            endpoint.delete()
    except Exception as e:
        print(e)

    # Delete the AutoML or Pipeline trainig job
    try:
        if "dag" in globals():
            dag.delete()
    except Exception as e:
        print(e)

    # Delete the custom trainig job
    try:
        if "job" in globals():
            job.delete()
    except Exception as e:
        print(e)

    # Delete the batch prediction job using the Vertex batch prediction object
    try:
        if "batch_predict_job" in globals():
            batch_predict_job.delete()
    except Exception as e:
        print(e)

    # Delete the hyperparameter tuning job using the Vertex hyperparameter tuning object
    try:
        if "hpt_job" in globals():
            hpt_job.delete()
    except Exception as e:
        print(e)

    if "BUCKET_NAME" in globals():
        ! gsutil rm -r $BUCKET_NAME